# Yelp API Calls
<p> In this project I am exploring the Yelp API to search my favorite city for cuisine types of my choice. At the end I will extract all results from my search and compile them into a dataframe. </p>

Import Modules

In [1]:
# Standard Imports
import pandas as pd
import os, json, math, os
from time import sleep
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

Load credentials

In [2]:
# Load API Creds
with open('creds.json') as file:
    login = json.load(file)
# Instantiate YelpAPI
yelp_api = YelpAPI(login['api-key'], timeout_s = 5.0)

Environment Variables

In [3]:
LOCATION = 'Fresno, CA, 93720'
TERM = 'Pizza'
JSON_FILE = f"Data/results_in_progress_pizza.json"

Create progress file if it doesn't exist already and set n_results

In [4]:
# Set default value to 0 results and no previous results
n_results = 0
previous_results = []
if os.path.isfile(JSON_FILE) == False:
    # In progress file doesn't exist
    if len(os.path.dirname(JSON_FILE)) > 0:
        os.makedirs(os.path.dirname(JSON_FILE), exist_ok = True)
    # Tell user new file was made
    print(f"[!] {JSON_FILE} not found. Saving empty list to file.")
    with open(JSON_FILE, 'w') as file:
        json.dump([], file)
else:
    # In Progress does exist
    print(f"[!] {JSON_FILE} already exists.") 
    with open(JSON_FILE, 'r') as file:
        previous_results = json.load(file)
        n_results = len(previous_results)

print(f"[!] {n_results} previous results found.")

[!] Data/results_in_progress_pizza.json already exists.
[!] 264 previous results found.


Get total results of API Search

In [5]:
# add offset to our original api call
results = yelp_api.search_query(location=LOCATION,
                                      term=TERM,
                                      offset = 20)
# Get data on the query size
total_results = results['total']
results_per_page = len(results['businesses'])

In [6]:
# Find out total pages
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
print(f"{int(n_pages)} pages with a total of {total_results} results")

1 pages with a total of 265 results


Add the current results to the JSON, then loop through the rest.

In [7]:
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as file:
     json.dump(previous_results, file)

In [8]:
# Not using TQDM - issues with mac m1
for i in range(1, n_pages + 1):
    sleep(.2)
    # check results length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    # save number of results for to use as offset
    n_results = len(previous_results)
    # use n_results as the OFFSET 
    results = yelp_api.search_query(location = LOCATION,
                                    term = TERM, 
                                    offset = n_results + 1)    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    with open(JSON_FILE,'w') as file:
        json.dump(previous_results, file)

Turn json into DataFrame

In [9]:
df = pd.read_json(JSON_FILE)
display(df.head(), df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,9ZURLR_1ys3lN3Li8E_C0w,heirloom-fresno-2,Heirloom,https://s3-media3.fl.yelpcdn.com/bphoto/yMIpE2...,False,https://www.yelp.com/biz/heirloom-fresno-2?adj...,1321,"[{'alias': 'newamerican', 'title': 'American (...",4.5,"{'latitude': 36.856482510116116, 'longitude': ...",[delivery],$$,"{'address1': '8398 N Fresno St', 'address2': '...",+15592289735,(559) 228-9735,2199.636351
1,_Sa-HeTVvUM4c-g9C3sucA,coast-2-coast-pizza-clovis-2,Coast 2 Coast Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/PvOD1x...,False,https://www.yelp.com/biz/coast-2-coast-pizza-c...,30,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 36.853642, 'longitude': -119.7279...","[pickup, delivery]",NaN,"{'address1': '1345 N Willow Ave', 'address2': ...",+15599007559,(559) 900-7559,3209.984511
2,wigKCWrJ2JotZtrVwW2nfQ,dominos-pizza-fresno-11,Domino's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/YpDm1T...,False,https://www.yelp.com/biz/dominos-pizza-fresno-...,30,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",3.0,"{'latitude': 36.871771, 'longitude': -119.7635...",[],$,"{'address1': '1170 E Champlain Dr', 'address2'...",+15594122010,(559) 412-2010,786.074774
3,vyXBP9lkiaPXP2MHU4PqDg,me-n-eds-coney-island-grill-fresno,Me-n-Ed's Coney Island Grill,https://s3-media3.fl.yelpcdn.com/bphoto/PWYBHb...,False,https://www.yelp.com/biz/me-n-eds-coney-island...,227,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"{'latitude': 36.850046, 'longitude': -119.788104}","[pickup, delivery]",$$,"{'address1': '175 E Via Del Parque', 'address2...",+15594401088,(559) 440-1088,2894.954943
4,TN4KQAfB2zHmB_KUC5wynQ,davincis-pizza-north-willow-fresno-4,DaVinci's Pizza - North Willow,https://s3-media2.fl.yelpcdn.com/bphoto/qgdTpd...,False,https://www.yelp.com/biz/davincis-pizza-north-...,46,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.5,"{'latitude': 36.8699013747264, 'longitude': -1...",[delivery],$,"{'address1': '7509 N Willow Ave', 'address2': ...",+15592989400,(559) 298-9400,3469.460183


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
279,xiPYoKcZ-QacPjrvUFRkpQ,the-annex-kitchen-fresno-2,The Annex Kitchen,https://s3-media1.fl.yelpcdn.com/bphoto/kng7_z...,False,https://www.yelp.com/biz/the-annex-kitchen-fre...,1323,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 36.80758, 'longitude': -119.83229}",[delivery],$$$,"{'address1': '2257 W Shaw Ave', 'address2': ''...",+15592488512,(559) 248-8512,8955.511895
280,sTQyBE8ZZOjdJzyGjG7Oyg,angelas-italian-restaurant-fresno,Angela's Italian Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/yqfW3w...,False,https://www.yelp.com/biz/angelas-italian-resta...,90,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 36.8131166270708, 'longitude': -1...","[delivery, pickup]",$$,"{'address1': '5251 N Blackstone Ave', 'address...",+15593745390,(559) 374-5390,6354.635022
281,LJl7jLPcHgBHJW0HrT5JbQ,round-table-pizza-fresno-10,Round Table Pizza,,False,https://www.yelp.com/biz/round-table-pizza-fre...,6,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.0,"{'latitude': 36.86768496257675, 'longitude': -...","[delivery, pickup]",NaN,"{'address1': '1512 E Champlain Dr', 'address2'...",+15594493125,(559) 449-3125,539.669072
282,I0ilCNy7TWxNw2au9VXsxA,pieology-pizzeria-fresno-9,Pieology Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/ZCIU2N...,False,https://www.yelp.com/biz/pieology-pizzeria-fre...,115,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 36.857257646354626, 'longitude': ...","[delivery, pickup]",$$,"{'address1': '8464 N Friant Rd', 'address2': '...",+15594128523,(559) 412-8523,2179.146509
283,SYJmPJ0XMi8PGkYWwrQdCQ,round-table-pizza-fresno-2,Round Table Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/VVAccA...,False,https://www.yelp.com/biz/round-table-pizza-fre...,81,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 36.821449469594015, 'longitude': ...","[delivery, pickup]",$$,"{'address1': '5702 N First St', 'address2': 'S...",+15594360100,(559) 436-0100,4908.170154


Save to CSV

In [10]:
df.to_csv('Data/final_results.csv.gz', compression = 'gzip', index = False)